In [266]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from glob import glob
from tqdm import tqdm
from torchmetrics.classification import BinaryAUROC
from sklearn.metrics import f1_score

In [247]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [344]:
class CicikNet(nn.Module):
    def __init__(self):
        super(CicikNet, self).__init__()

        # First 2D convolutional layer, taking in 1 input channel (image),
        # outputting 32 convolutional features, with a square kernel size of 3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1,padding=1)
        # Second 2D convolutional layer, taking in the 32 input layers,
        # outputting 64 convolutional features, with a square kernel size of 3
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=5, stride=1,padding=2)

        # Designed to ensure that adjacent pixels are either all 0s or all active
        # with an input probability
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)

        # First fully connected layer
        self.fc1 = nn.Linear(64*64+64*64, 128)
        # Second fully connected layer that outputs our 10 labels
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        x1 = torch.flatten(self.conv1(x),1)
        x2 = torch.flatten(self.conv2(x),1)
        
        x = torch.cat([x1,x2],dim=1)

        x = self.fc1(x)
        x = F.relu(x)

        
        x = self.fc2(x)

        output = torch.sigmoid(x)
        return output

net = CicikNet().to(device)
# print(net)

In [345]:
def RetinaTransform(image):
    # Bringing image to 0-1 range, #ATTENTION: CHECK IF ALL IMAGES min max is 0 and 255
    image = image / 255
    
    
    image = resize(image, (64,64),anti_aliasing=False) # resizing image because original does not fit in memory.
    
    
    # Bringing data to CHW format
    # N is a batch size, C denotes a number of channels, 
    # H is a height of input planes in pixels, and W is width in pixels.
    image = image.transpose([2,0,1])
    
    #Fixing dtype to avoid runtime error and save memory
    image = torch.tensor(image ,dtype=torch.float32)
    
    return image

In [346]:
class RetinaDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, label_path, data_folder, transform=None):
        
        self.label_frame = pd.read_csv(label_path)
        self.data_folder = data_folder
        self.transform = transform
        
        # Load dataset into memory
        self.cuda_image_array = []
        for image_name in tqdm(glob(self.data_folder+"/*")):
            image = io.imread(image_name)

            label_frame_index = int(image_name.split("/")[-1].split(".")[0])
            label = self.label_frame[self.label_frame["ID"]==label_frame_index]["Disease_Risk"].values[0]

            image = self.transform(image)
            label =  torch.tensor(label ,dtype=torch.float32)

            self.cuda_image_array.append((image.to(device),label.to(device)))
        
        
    def __len__(self):
        return self.label_frame.shape[0]

    def __getitem__(self, idx):

        return self.cuda_image_array[idx]

In [347]:
# retina_dataset = RetinaDataset(data_folder = "Evaluation_Set/Validation", label_path = "Evaluation_Set/RFMiD_Validation_Labels.csv", transform = RetinaTransform)

In [348]:
# retina_dataloader = DataLoader(retina_dataset, batch_size=256, shuffle=False, num_workers=0)

In [354]:
EPOCHS = 1000
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
bauc = BinaryAUROC(thresholds=None)

In [355]:
# MODEL learns the trivial case of all 1s, which means it has capacity to learn:

for epoch in range(EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    
    
    predictions = []
    ground_truth = []
    for idx, data in enumerate(retina_dataloader, 0):
        # get the inputs
        # print(i,data)
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs.flatten(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        predictions.append(outputs.flatten())
        ground_truth.append(labels.flatten())
        
        
    predictions=torch.cat(predictions)
    ground_truth = torch.cat(ground_truth)
    
    
    healthy_f1 = f1_score((predictions>0.5).float().cpu(),ground_truth.cpu(),pos_label=0)
    rocauc = bauc(predictions,ground_truth)
    print('[%d, %5d] loss: %.3f, ROC: %.3f, F1_healthy: %.3f' % (epoch + 1, idx + 1, running_loss, rocauc, healthy_f1))
    
# print('Finished Training')

[1,     3] loss: 1.490, ROC: 0.691, F1_healthy: 0.000
[2,     3] loss: 1.490, ROC: 0.691, F1_healthy: 0.000
[3,     3] loss: 1.489, ROC: 0.691, F1_healthy: 0.000
[4,     3] loss: 1.489, ROC: 0.692, F1_healthy: 0.000
[5,     3] loss: 1.488, ROC: 0.692, F1_healthy: 0.000
[6,     3] loss: 1.488, ROC: 0.693, F1_healthy: 0.000
[7,     3] loss: 1.487, ROC: 0.693, F1_healthy: 0.000
[8,     3] loss: 1.486, ROC: 0.694, F1_healthy: 0.000
[9,     3] loss: 1.486, ROC: 0.694, F1_healthy: 0.000
[10,     3] loss: 1.485, ROC: 0.694, F1_healthy: 0.000
[11,     3] loss: 1.485, ROC: 0.695, F1_healthy: 0.000
[12,     3] loss: 1.484, ROC: 0.695, F1_healthy: 0.000
[13,     3] loss: 1.483, ROC: 0.696, F1_healthy: 0.000
[14,     3] loss: 1.483, ROC: 0.696, F1_healthy: 0.000
[15,     3] loss: 1.482, ROC: 0.697, F1_healthy: 0.000
[16,     3] loss: 1.481, ROC: 0.697, F1_healthy: 0.000
[17,     3] loss: 1.480, ROC: 0.698, F1_healthy: 0.000
[18,     3] loss: 1.480, ROC: 0.698, F1_healthy: 0.000
[19,     3] loss: 1

In [356]:
# f1_score((predictions>0.5).float().cpu(),ground_truth.cpu(),pos_label=0)